In [26]:
import requests
import pandas as pd
import time

In [27]:
#input video url
#referer = input("Enter referer: ")

#test url
referer = 'https://www.tiktok.com/@toanchodien23/video/7121230509027478810'

#get video id from referer - example: 7121230509027478810
id = referer.split('/')[5]


In [36]:
#Create DataFrame to store comments
df = pd.DataFrame(columns=['cid', 'text', 'time', 'digg_count', 'reply_comment_total', 'user_id', 'user_nickname', 'user_unique_id', 'referer'])

#cid: comment id
#text: comment
#time: comment time
#digg_count: number of likes
#reply_comment_total: number of replies
#user_id: user id
#user_nickname: user nickname
#user_unique_id: user unique id
#referer: video url

In [29]:
#Epoch & Unix Timestamp Conversion to GMT+0 time
#Cuz Tiktok API return Epoch & Unix Timestamp, we need to convert it to GMT+0 time
def epoch_to_date(epoch):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(epoch))

#If you want to convert it to local time, use this function
# def epoch_to_date(epoch):
#     return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(epoch))

In [38]:
cookies = {
    '_ttp': '29R1dUogCz9evh5bjMrLFIxQJ3L',
    's_v_web_id': 'verify_l4ct1juv_06cHWVBK_PaGW_4Daa_9ZKq_NoNxlIx4BD2W',
    'tt_csrf_token': 'iztXcztc-odZDYbduwcZE8q_Pr04KVA0Iq_0',
    'csrf_session_id': '5ceea1925aa09df79cacdf7da670460b',
    'passport_csrf_token': '94175df818e9485d08df6a1fcd70ea6b',
    'passport_csrf_token_default': '94175df818e9485d08df6a1fcd70ea6b',
    'd_ticket': '1f4b47f14971d6f2d9dab12eac466dea9cb6b',
    'passport_auth_status': '413b8fdc2e94511cd587f963e312de3a%2C',
    'passport_auth_status_ss': '413b8fdc2e94511cd587f963e312de3a%2C',
    'sid_guard': '076bd311dc7a4f00da0f5cf854ec9931%7C1661858152%7C5184000%7CSat%2C+29-Oct-2022+11%3A15%3A52+GMT',
    'uid_tt': '609bfb461acb534d23bbc9c8c2e05679c64930e0152a1a521ccac0e72e0d1970',
    'uid_tt_ss': '609bfb461acb534d23bbc9c8c2e05679c64930e0152a1a521ccac0e72e0d1970',
    'sid_tt': '076bd311dc7a4f00da0f5cf854ec9931',
    'sessionid': '076bd311dc7a4f00da0f5cf854ec9931',
    'sessionid_ss': '076bd311dc7a4f00da0f5cf854ec9931',
    'sid_ucp_v1': '1.0.0-KDhlZTc5MWFkNTFhN2Q4ZmVhNTZjZmE5OTcyY2E5MzQyNjIwZDczYjAKIAiviMHE15jl_2EQ6OK3mAYYswsgDDDpuf6PBjgCQPEHEAMaBm1hbGl2YSIgMDc2YmQzMTFkYzdhNGYwMGRhMGY1Y2Y4NTRlYzk5MzE',
    'ssid_ucp_v1': '1.0.0-KDhlZTc5MWFkNTFhN2Q4ZmVhNTZjZmE5OTcyY2E5MzQyNjIwZDczYjAKIAiviMHE15jl_2EQ6OK3mAYYswsgDDDpuf6PBjgCQPEHEAMaBm1hbGl2YSIgMDc2YmQzMTFkYzdhNGYwMGRhMGY1Y2Y4NTRlYzk5MzE',
    'store-idc': 'useast2a',
    'store-country-code': 'vn',
    'store-country-code-src': 'uid',
    'tt-target-idc': 'alisg',
    'cmpl_token': 'AgQQAPOFF-RMpbHBemLwop0W-MbaVVbI_4AOYMU5HQ',
    '__tea_cache_tokens_1988': '{%22user_unique_id%22:%227108724646376621570%22%2C%22timestamp%22:1661858807046%2C%22_type_%22:%22default%22}',
    '_abck': '8197FC42F83990B3157D81C84D21BFD5~-1~YAAQLlBNGy+pFP2CAQAApc0yEwh5THB5lqykWDfDuOSnDhLENnWneOCctfKK79MQKLhRkfcyyL6FkvTeoe5oheL9BB5Wnm0DtkbRhfsxKXEp6CrT59JKEdG7foqhO2tHOoDjIxFxR/eD4VCSGAH/ZsYt4nKeLPukfUSr8qo3a718HD6LEl2LqJ/AQVcbPTqDc7yIP/t+8l6RtzON/VWGPCk2kjEDkkuIfCR9iuxSB6VbQiyzGY64f0PN57pTO06f57DhCWkWrsRbDaEJU13q8KnCY+SIGpWzNjBHLe5hgJkHFPZ61c5gLg9POWo8h74C9zLHyCFjO00lYNxXKREokNJ2jBcI3ayep3YbBnXQKikltoEiaVv66AUu75t4hIp+F0mSonYueAoLnF4=~-1~-1~-1',
    'bm_sz': '404AF1D62F9E243D5F6FCD16F1D79D33~YAAQLlBNGzCpFP2CAQAApc0yExElRwaYOIcXoiKgwpmRjn9dKEWmsDY2Dm27YQhFAC5ishW/4ypvHZTW6E0461o3nBK9aLv88rKHnkPzLjX5bJGykjtUQJemguJ2EH3RHNyGZNsfFeUrZrjUCB5v13qfJbQxqXd7YSxpbbMMDO6raS5Vf0amMXJXVLX0NfEXl3PuYmmYMDCy1ux/tPy2A2Y5CympMcMyeH4eK2TZ0yF5PkhG9C/YPPjyHvQNOHrjRvXgPseYlMQ2V+H3U3BF4g+CYKU5Zftzw3CfyZPddjTZHU8=~3291184~3552069',
    'ak_bmsc': '2870BDC16E4584CF994EC4C65A327156~000000000000000000000000000000~YAAQLlBNG1WpFP2CAQAAGPsyExHF/yR1s4mRHxBNig7s+Y/e5JqtPdJ9tAaYPlPrbyEMaFptEnx2jGPtwWiiMGUOaS8i+4oh7iQ2xG52DDihaYxPTM+SgAbk6ews7wcqRRUiHibRy0m4bE/HNLBdeXM66iN/M/46XkoA7PwLjuNYwWYAVn3/zidWyE/Vl8RiqEElDCGxxm9YsQ70O6rBUX2dXWOXwMtYnH5jVL8sWmTF0yRUmpfla3If6m+3R/eRzyzNO/qc2Wt5laZC6nS7xCCQdQdIOd2SF/f4efS4NcXJwk2Q25+cLND2VDq6hNi1d9X7U+j35tYGU8ZCYuH0SbcNu8tVOVlvTSCpsCi0qDpP/iNo9VI2DHmh/xTR0JHfwZXFfG0Oa4W9VfSG',
    'ttwid': '1%7CqDEYDh1xjNzhQymfMQXUBbiZ0avlnad6c477HMaAjz4%7C1662474926%7C9ac339ad183588a6710e9ff9e338c8b891cd14f0179bd579c5b7397f51eaece7',
    'bm_sv': '545CEA156158F5926E3294D27CFFB99A~YAAQHlBNG+g9EP2CAQAArNdCExGvP4b6UtG260o4R1qybCJRXt3Mt+93/f7AprmvInFBjj+5pPUWfS45FHT0LMGmhSwaeAHWIn1+DNGCof10Bx3ZuqzgYQb+MPoSxa0kaMq/pQisAZRSkMY9+40MokluAriz2KVsnzLNrkzUp15EigyB0b9JeH6kAFslDuWwLIkqznDsUDHKRnoviOeW1IYhAR3S4LJxRWO1HfdMQ9GloRwyh3OObmtcYJAtVcYqwA==~1',
    'odin_tt': '1cb27cbeb3f6042a9a1899c374811d708630d0ba434143beec52c62294b46376d428d7a9130836098c93371be779f90c68f8a03ab6d4b96964ae09e45eaa713e',
    'msToken': 'wEZfUhNWpniGXRo_xBGKcmno5MKHejayplzCiD9uifvSZ2-Du763DjS1q8iyR03uUlrAWwwlWDci_opptAVa7acQr165fNciR2A2pxcnyT67APgyYHAW-GJ6AOT3j60yJ7ISYCk=',
    'msToken': 'wEZfUhNWpniGXRo_xBGKcmno5MKHejayplzCiD9uifvSZ2-Du763DjS1q8iyR03uUlrAWwwlWDci_opptAVa7acQr165fNciR2A2pxcnyT67APgyYHAW-GJ6AOT3j60yJ7ISYCk=',
    'passport_fe_beating_status': 'true',
}

headers = {
    'authority': 'www.tiktok.com',
    'accept': '*/*',
    'accept-language': 'en,vi;q=0.9',    
    'referer': referer,
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36 Edg/105.0.1343.27',
}

#set cursor - address of the first row
cursor = 0

#set count - number of rows to read - maximum 50 rows per request
count = 50

In [42]:
#Cuz TikTok API only return {count} comments per request, so we need to loop to get all comments
f = 0
while True:
    f += 1
    try:
        response = requests.get(f'https://www.tiktok.com/api/comment/list/?aweme_id={id}&count={count}&cursor={cursor}', headers=headers)
        json_data = response.json()
        cursor = json_data['cursor']
        for data in json_data['comments']:
            #append data to dataframe
            df = df.append({'cid':data['cid'], 'text':data['text'], 'time':epoch_to_date(data['create_time']), 'digg_count':data['digg_count'], 'reply_comment_total':data['reply_comment_total'], 'user_id':data['user']['uid'], 'user_nickname':data['user']['nickname'], 'user_unique_id':data['user']['unique_id'], 'referer':referer}, ignore_index=True)
            f += 1
    except:
        print(f"Error at {f}th request")
        break

C:\Users\Admin\AppData\Local\Temp\ipykernel_15944\561467865.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'cid':data['cid'], 'text':data['text'], 'time':epoch_to_date(data['create_time']), 'digg_count':data['digg_count'], 'reply_comment_total':data['reply_comment_total'], 'user_id':data['user']['uid'], 'user_nickname':data['user']['nickname'], 'user_unique_id':data['user']['unique_id'], 'referer':referer}, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_15944\561467865.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'cid':data['cid'], 'text':data['text'], 'time':epoch_to_date(data['create_time']), 'digg_count':data['digg_count'], 'reply_comment_total':data['reply_comment_total'], 'user_id':data['user']['uid'], 'user_nickname':data['user']['nickname'], '

Error at 5081th request


In [43]:
#Dataframe description
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4980 entries, 0 to 4979
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   cid                  4980 non-null   object
 1   text                 4980 non-null   object
 2   time                 4980 non-null   object
 3   digg_count           4980 non-null   object
 4   reply_comment_total  4980 non-null   object
 5   user_id              4980 non-null   object
 6   user_nickname        4980 non-null   object
 7   user_unique_id       4980 non-null   object
 8   referer              4980 non-null   object
dtypes: object(9)
memory usage: 350.3+ KB


,cid,text,time,digg_count,reply_comment_total,user_id,user_nickname,user_unique_id,referer
0,7121240491988288283,lâu quá mới thấy a Toàn ăn bằng miệng,2022-07-17 07:30:23,9102,127,6625938655943294978,Gia Bảo,bao31_10,https://www.tiktok.com/@toanchodien23/video/71...
1,7121230961790010138,:)) ẩu rồi đó toàn,2022-07-17 06:53:17,8011,13,6876679968669189121,xuantrong,xuantronggg,https://www.tiktok.com/@toanchodien23/video/71...
2,7121232772936647451,Nhìn lâu giống Park see ro yi,2022-07-17 07:00:18,3734,57,7022776734878958619,Gút bai mai lớp,motdemsaye,https://www.tiktok.com/@toanchodien23/video/71...
3,7121231101334913819,MV tỉ đô,2022-07-17 06:53:52,2254,3,6976890446857831426,Kiên vui vẻ,kienvuive28,https://www.tiktok.com/@toanchodien23/video/71...
4,7121233348383310618,Khứa này nhiều conten thật 😂😂😂,2022-07-17 07:02:36,1352,24,7110155692086084610,Baodeptrai113,baodeptrais,https://www.tiktok.com/@toanchodien23/video/71...


In [ ]:
#save to csv
df.to_csv('TikTok_comments.csv', index=False)